In [1]:
import scipy.stats as sps
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

Set search and chain windows, and minimum chain length.

In [2]:
chain_win_start = 7 * 24 * 3600
search_win_end = 10 * 24 * 3600
min_len = 5

Read simulated data from file.

In [3]:
runs = []
nchains = 0
with open('simulated_data_2w_big.txt', 'r') as f:
    for rs in f:
        rl = rs.split(',')
        r = (int(rl[0]), int(rl[1]), int(rl[2]), int(rl[3]), int(rl[4]), int(rl[5]))
        nchains = max(nchains, r[5] + 1)
        if r[3] <= search_win_end and r[4] == 0:
            runs.append(r)

Number of simulated chains:

In [4]:
nchains

500000

Number of simulated runs that appear in the data set:

In [5]:
len(runs)

8026763

Run the temporal chaining algorithm.

In [6]:
def links(r1, r2):
    return r1[1] + r1[2] == r2[1] and r2[3] - r2[2] >= r1[3]

In [7]:
# build dictionaries of runs for fast lookup
chain_data = {}
search_data = {}
for r in runs:
    if r[3] >= chain_win_start:
        if r[1] in chain_data:
            chain_data[r[1]].append(r)
        else:
            chain_data[r[1]] = [r]

    else:
        target_exp = r[1] + r[2]
        if target_exp in search_data:
            search_data[target_exp].append(r)
        else:
            search_data[target_exp] = [r]

In [8]:
# sieves out a chain starting with run i. Deletes all forward-colliding runs
# and their chains        
def chain(exp, k = 0):
    source_runs = chain_data.get(exp, [])
    if len(source_runs) >= k + 1:
        ch = [source_runs.pop(k)]
    else:
        return []
   
    target_exp = ch[0][1] + ch[0][2]
    target_runs = []
    for k, r in enumerate(chain_data.get(target_exp, [])):
        if links(ch[0], r):
            target_runs.append(k)
    
    if len(target_runs) == 1:
        return ch + chain(target_exp, target_runs[0])
    elif len(target_runs) > 1:
        for i, k in enumerate(target_runs):
            chain(target_exp, k - i)
        return ch
    else:
        return ch

In [ ]:
# create chains truncating after forward-collisions. Discard short chains.
cand_chains = []
exp_keys = list(chain_data.keys())
exp_keys.sort()

n = len(exp_keys)
while n > 0:
    ch = chain(exp_keys[0])
    if len(ch) > 0:
        if len(ch) >= min_len:
            cand_chains.append(ch)
        r = ch[-1]
        target_exp = r[1] + r[2]
        if target_exp in search_data:
            search_data[target_exp].append(r)
        else:
            search_data[target_exp] = [r]
    else:
        exp_keys.pop(0)
        n = len(exp_keys)
        continue

Diagnose chain integrity before back-collision pruning:

In [ ]:
players_per_chain = np.zeros(len(cand_chains), dtype = int)
chains_per_player = np.zeros(nchains, dtype = int)
chain_lens = np.zeros(len(cand_chains), dtype = int)
nruns = 0

for i, ch in enumerate(cand_chains):
    nruns += len(ch)
    players = set()
    for r in ch:
        players.add(r[5])
    players_per_chain[i] = len(players)
    
    for p in players:
        chains_per_player[p] += 1
        
    chain_lens[i] = len(ch)

In [ ]:
plt.title('Players Per Chain')
sns.countplot(x = players_per_chain, color = 'b')

In [ ]:
plt.figure(figsize = (10, 4))
plt.title('Chains Per Player')
sns.countplot(x = chains_per_player, color = 'b')

In [ ]:
plt.figure(figsize = (10, 4))
plt.title('Chain Lengths')
sns.countplot(x = chain_lens, color = 'b')

In [ ]:
(players_per_chain > 1).sum() / len(cand_chains)

In [ ]:
sps.spearmanr(chain_lens, players_per_chain)

Number of chains that need to be checked for back-collisions:

In [ ]:
n = len(cand_chains)
n

Total number of runs:

In [ ]:
nruns

Perform back-collision pruning.

In [ ]:
# transfer isolated runs in the chaining dict to the search dict
for r_arr in chain_data.values():
    for r in r_arr:
        target_exp = r[1] + r[2]
        if target_exp in search_data:
            search_data[target_exp].append(r)
        else:
            search_data[target_exp] = [r]

In [ ]:
# prune chains after back-collisions
for i in range(n):
    ch = cand_chains[i]
    for j in range(1, len(ch)):
        trunc = False
        colls = search_data.get(ch[j][1])
        if colls == None: 
            continue

        for r in colls:
            if links(r, ch[j]):
                trunc = True
                break

        if trunc:
            cand_chains[i] = ch[:j]
            break

In [ ]:
# delete short chains
i = 0
while i < len(cand_chains):
    if len(cand_chains[i]) < min_len:
        cand_chains.pop(i)
    else:
        i += 1

Diagnose chain integrity after back-collision pruning.

In [ ]:
players_per_chain = np.zeros(len(cand_chains), dtype = int)
chains_per_player = np.zeros(nchains, dtype = int)
chain_lens = np.zeros(len(cand_chains), dtype = int)
nruns = 0

for i, ch in enumerate(cand_chains):
    nruns += len(ch)
    players = set()
    for r in ch:
        players.add(r[5])
    players_per_chain[i] = len(players)
    
    for p in players:
        chains_per_player[p] += 1
        
    chain_lens[i] = len(ch)

In [ ]:
plt.title('Players Per Chain')
sns.countplot(x = players_per_chain, color = 'b')

In [ ]:
plt.figure(figsize = (10, 4))
plt.title('Chains Per Player')
sns.countplot(x = chains_per_player, color = 'b')

In [ ]:
plt.figure(figsize = (10, 4))
plt.title('Chain Lengths')
sns.countplot(x = chain_lens, color = 'b')

In [ ]:
(players_per_chain > 1).sum() / len(cand_chains)

In [ ]:
sps.spearmanr(chain_lens, players_per_chain)

Total remaining runs:

In [ ]:
nruns